# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [18]:
import pandas as pd
import numpy as np
import regex as re
from nltk.tokenize import word_tokenize
import nltk
# nltk.download('punkt')
from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.probability import FreqDist
import random

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/estelaoshiro/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/estelaoshiro/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/estelaoshiro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# your code here


In [3]:
data = pd.read_csv("training.16.csv", encoding='iso-8859-1')

In [4]:
data

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [5]:
data2 = data.sample(20000)
data2.shape

(20000, 6)

In [6]:
data2.columns = ['target', 'id', 'date', 'flag','user','text']
data2.head()

,target,id,date,flag,user,text
1438592,4,2061364163,Sat Jun 06 20:42:51 PDT 2009,NO_QUERY,peseypich,perfect date day...natural history museum oohs...
1071981,4,1966465798,Fri May 29 17:49:05 PDT 2009,NO_QUERY,prema94,@marco_alan I'm sorry. =\ well it will get bet...
776580,0,2322187485,Wed Jun 24 22:11:27 PDT 2009,NO_QUERY,ReinaRobinson,@Wale Got Damn You!!! haha! You know you seen ...
1502455,4,2071705431,Sun Jun 07 19:48:06 PDT 2009,NO_QUERY,TheMystifyer,@NewEnglandBaby saw that you were following me...
888239,4,1687218367,Sun May 03 08:06:01 PDT 2009,NO_QUERY,kel7alpha,@deanomeano Awwww. Thanks for clueing me in o...


In [7]:
data2['is_positive'] = data2['target'].replace({0: 0, 4: 1})
data2.drop('target', axis=1, inplace=True)


In [8]:
data2.head()

,id,date,flag,user,text,is_positive
1438592,2061364163,Sat Jun 06 20:42:51 PDT 2009,NO_QUERY,peseypich,perfect date day...natural history museum oohs...,1
1071981,1966465798,Fri May 29 17:49:05 PDT 2009,NO_QUERY,prema94,@marco_alan I'm sorry. =\ well it will get bet...,1
776580,2322187485,Wed Jun 24 22:11:27 PDT 2009,NO_QUERY,ReinaRobinson,@Wale Got Damn You!!! haha! You know you seen ...,0
1502455,2071705431,Sun Jun 07 19:48:06 PDT 2009,NO_QUERY,TheMystifyer,@NewEnglandBaby saw that you were following me...,1
888239,1687218367,Sun May 03 08:06:01 PDT 2009,NO_QUERY,kel7alpha,@deanomeano Awwww. Thanks for clueing me in o...,1


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [10]:
def clean_up(s):
    """
    Cleans up numbers, URLs, and special characters from a string.

    Args:
        s: The string to be cleaned up.

    Returns:
        A string that has been cleaned up.
    """
    s = str(s).lower()
    s = re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', ' ' , s)
    s = s.replace("\\","").replace("_"," ")
    s = re.sub(r'\W+',' ',s) # Replace everything non-alpahnumeric by ' '
    s = re.sub(r'\s+',' ',s) # Replace one or more whitespaces by  ' '
    s = re.sub(r'\d+',' ',s) # Replace one or more digits by  ' '
    s = re.sub(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+)'," ", s) # Replace e-mails by ''
    s = s.replace(' br ',' ')

    return s

def tokenize(s):
    """
    Tokenize a string.

    Args:
        s: String to be tokenized.

    Returns:
        A list of words as the result of tokenization.
    """
    text = word_tokenize(s)
    return text

def stem_and_lemmatize(l):
    """
    Perform stemming and lemmatization on a list of words.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after being stemmed and lemmatized.
    """
    lemmatizer = WordNetLemmatizer()
    lem = [lemmatizer.lemmatize(w) for w in l]
    stemmer = PorterStemmer()
    ste = [stemmer.stem(w) for w in l]
    return (lem)

def remove_stopwords(l):
    """
    Remove English stopwords from a list of strings.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after stop words are removed.
    """
    stop_words = list(stopwords.words('english')) 
    stop_words
    new_list = [w for w in l if w not in stop_words]
    return new_list

In [11]:
data2['text_processed'] = data2['text'].apply(clean_up)
data2['text_processed'] = data2['text_processed'].apply(tokenize)
data2['text_processed'] = data2['text_processed'].apply(stem_and_lemmatize)
data2['text_processed'] = data2['text_processed'].apply(remove_stopwords)

data2.reset_index()
data2

,id,date,flag,user,text,is_positive,text_processed
1438592,2061364163,Sat Jun 06 20:42:51 PDT 2009,NO_QUERY,peseypich,perfect date day...natural history museum oohs...,1,"[perfect, date, day, natural, history, museum,..."
1071981,1966465798,Fri May 29 17:49:05 PDT 2009,NO_QUERY,prema94,@marco_alan I'm sorry. =\ well it will get bet...,1,"[marco, alan, sorry, well, get, better, much, ..."
776580,2322187485,Wed Jun 24 22:11:27 PDT 2009,NO_QUERY,ReinaRobinson,@Wale Got Damn You!!! haha! You know you seen ...,0,"[wale, got, damn, haha, know, seen, u, corner,..."
1502455,2071705431,Sun Jun 07 19:48:06 PDT 2009,NO_QUERY,TheMystifyer,@NewEnglandBaby saw that you were following me...,1,"[newenglandbaby, saw, following, figured, foll..."
888239,1687218367,Sun May 03 08:06:01 PDT 2009,NO_QUERY,kel7alpha,@deanomeano Awwww. Thanks for clueing me in o...,1,"[deanomeano, awwww, thanks, clueing]"
...,...,...,...,...,...,...,...
125548,1834337166,Mon May 18 02:32:57 PDT 2009,NO_QUERY,lee_robson,"However, I have had no breakfast...",0,"[however, breakfast]"
835925,1558342603,Sun Apr 19 08:04:31 PDT 2009,NO_QUERY,emilymccue,@andymcintyre05 second what hayes said,1,"[andymcintyre, second, hayes, said]"
550034,2202807960,Tue Jun 16 22:27:16 PDT 2009,NO_QUERY,ScorpioSwag,somehow missed @songzyuuup live AGAIN!! watch...,0,"[somehow, missed, songzyuuup, live, watching, ..."
1399010,2054179456,Sat Jun 06 06:45:21 PDT 2009,NO_QUERY,melaniestarship,"gunna hit the hay, goodnight interwebz!",1,"[gunna, hit, hay, goodnight, interwebz]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [12]:
# your code here

In [13]:
fdist = FreqDist()
for sentence in data2['text_processed']:
    for word in sentence:
        fdist[word] += 1
fdist

FreqDist({'wa': 1379, 'day': 1375, 'good': 1164, 'get': 1040, 'go': 1017, 'like': 976, 'love': 912, 'got': 908, 'quot': 881, 'u': 873, ...})

In [14]:
most_freq_words = fdist.most_common(5000)
most_freq_words

[('wa', 1379),
 ('day', 1375),
 ('good', 1164),
 ('get', 1040),
 ('go', 1017),
 ('like', 976),
 ('love', 912),
 ('got', 908),
 ('quot', 881),
 ('u', 873),
 ('today', 869),
 ('work', 855),
 ('going', 820),
 ('time', 817),
 ('lol', 758),
 ('one', 748),
 ('back', 730),
 ('know', 660),
 ('really', 627),
 ('im', 618),
 ('new', 594),
 ('amp', 590),
 ('see', 586),
 ('want', 566),
 ('night', 566),
 ('home', 545),
 ('need', 531),
 ('thanks', 529),
 ('ha', 529),
 ('well', 518),
 ('na', 512),
 ('oh', 508),
 ('think', 507),
 ('still', 497),
 ('miss', 478),
 ('much', 459),
 ('last', 458),
 ('twitter', 456),
 ('feel', 455),
 ('great', 443),
 ('morning', 418),
 ('tomorrow', 414),
 ('make', 412),
 ('hope', 412),
 ('haha', 390),
 ('fun', 386),
 ('wish', 382),
 ('right', 380),
 ('week', 371),
 ('sad', 364),
 ('sleep', 362),
 ('bad', 359),
 ('would', 336),
 ('thing', 335),
 ('tonight', 332),
 ('friend', 331),
 ('nice', 330),
 ('better', 329),
 ('come', 327),
 ('sorry', 319),
 ('yeah', 313),
 ('could', 30

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [15]:
# your code here

In [16]:
col=[]
for i in most_freq_words:
   col.append(i[0])
print(col)

['wa', 'day', 'good', 'get', 'go', 'like', 'love', 'got', 'quot', 'u', 'today', 'work', 'going', 'time', 'lol', 'one', 'back', 'know', 'really', 'im', 'new', 'amp', 'see', 'want', 'night', 'home', 'need', 'thanks', 'ha', 'well', 'na', 'oh', 'think', 'still', 'miss', 'much', 'last', 'twitter', 'feel', 'great', 'morning', 'tomorrow', 'make', 'hope', 'haha', 'fun', 'wish', 'right', 'week', 'sad', 'sleep', 'bad', 'would', 'thing', 'tonight', 'friend', 'nice', 'better', 'come', 'sorry', 'yeah', 'could', 'getting', 'though', 'look', 'happy', 'way', 'say', 'hate', 'watching', 'wait', 'bed', 'people', 'gon', 'weekend', 'guy', 'school', 'thank', 'tweet', 'next', 'show', 'x', 'even', 'life', 'awesome', 'hey', 'take', 'wan', 'soon', 'yes', 'dont', 'hour', 'let', 'girl', 'long', 'year', 'sick', 'cant', 'never', 'working', 'please', 'first', 'everyone', 'lt', 'best', 'cool', 'movie', 'ok', 'find', 'done', 'looking', 'ever', 'tired', 'watch', 'little', 'another', 'lot', 'feeling', 'phone', 'already'

In [27]:

col = set([word for tweet in data2['text_processed'] for word in tweet])
matrix = np.zeros((len(data2), len(col)), dtype=int)

for i, tweet in enumerate(data2['text_processed']):
    for word in tweet:
        if word in col:
            matrix[i, list(col).index(word)] = 1


In [28]:
dataset = [({word: (word in tweet) for word in col}, label) for tweet, label in zip(data2['text_processed'], data2['is_positive'])]


### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [ ]:
# your code here

In [29]:

data4 = [({word: (word in tweet) for word in col}, 'is_positive') for tweet in (data2['text_processed'])]

random.shuffle(data4)

# split - training and testing sets
split_ratio = 0.8
split_index = int(len(data4) * split_ratio)
train_data = data4[:split_index]
test_data = data4[split_index:]

# train Naive Bayes classifier
classifier = nltk.NaiveBayesClassifier.train(train_data)

# most informative features
classifier.show_most_informative_features()



Most Informative Features


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [ ]:
# your code here

In [30]:
accuracy = nltk.classify.accuracy(classifier, test_data)
print("Accuracy:", accuracy)

Accuracy: 1.0


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here